# 🏠 Homework: Predicting Airbnb Listing Prices

**Your mission:** Clean and analyze a real-world Airbnb dataset, and build a model to predict listing prices.

This dataset contains dirty, messy, and surprising data — just like real life. Your job is to:

1. **Explore and clean the data**
2. **Engineer meaningful features**
3. **Split into train/test sets**
4. **Build and evaluate a linear regression model**
5. **Interpret your results**


## 🔍 Part A: Load and Explore

1. Load the CSV file `airbnb_dirty_mock.csv`
2. View column names, data types, and general structure
3. Plot a histogram of `price`
4. What issues or red flags do you immediately notice?

In [ ]:
import os
import time
import matplotlib.pyplot as plt
import pandas as pd
import pgeocode
import numpy as np
from sklearn.model_selection import train_test_split, cross_validate

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder

print(f"NumPy version: {np.__version__}")
print(f"Pandas version: {pd.__version__}")

# This is to avoid the SettingWithCopyWarning in pandas
# https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
pd.options.mode.copy_on_write = True

In [ ]:


csv_url = 'https://www.dropbox.com/scl/fi/gvg9vt3lcnh1807q0enlm/airbnb_dirty.csv?rlkey=x2q13wifq69djvu85zmsof530&dl=1'
csv_file = 'airbnb_dirty.csv'

if not os.path.exists(csv_file):
    !wget -O "$csv_file" "$csv_url"


df = pd.read_csv(csv_file)

display(df)

In [ ]:
print(df.columns, df.shape)

df.describe()

In [ ]:
price_series = df['price'].str.extract(r'(\d+\.?\d*)', expand=False).astype(float).dropna()

# --- Plotting a Box Plot to Visualize Outliers ---
plt.figure(figsize=(12, 4))
plt.boxplot(price_series, vert=False)
plt.title('Box Plot of Airbnb Prices')
plt.xlabel('Price ($)')
plt.yticks([]) # Hide y-axis ticks for clarity
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.savefig('price_boxplot.png')

In [ ]:
p9999 = price_series.quantile(0.9999)
capped_prices = price_series[price_series <= p9999]

# Use a standard rule (e.g., Sturges' rule) for a reasonable number of bins
# num_bins_capped = int(np.ceil(1 + np.log2(len(capped_prices))))


# --- Plotting a More Informative Capped Histogram ---
plt.figure(figsize=(12, 6))
plt.hist(capped_prices, bins=10, edgecolor='black')
plt.title(f'Distribution of Airbnb Prices (Capped at 99th percentile: ${p9999:.2f})')
plt.xlabel('Price ($)')
plt.ylabel('Frequency')

# Add lines for mean and median to show skewness
plt.axvline(price_series.median(), color='red', linestyle='dashed', linewidth=2, label=f'Median Price: ${price_series.median():.2f}')
plt.axvline(price_series.mean(), color='green', linestyle='dashed', linewidth=2, label=f'Mean Price: ${price_series.mean():.2f}')

plt.legend()
plt.grid(axis='y', alpha=0.75)
plt.savefig('price_histogram_capped.png')

## 🧹 Part B: Data Cleaning

**Clean your dataset to make it ready for modeling.** Consider the following steps:
- Fix or convert `price` to numeric
- Convert `last_review` to datetime (hint: use `errors='coerce'`)
- Handle invalid or missing data in `reviews_per_month`
- Drop columns that are not useful (e.g., `host_name`, `host_phone`, or duplicates)
- Handle duplicate rows
- Make sure categorical values are consistent (e.g. whitespace in `room_type`)

In [ ]:
# Remove " USD" and "$" using a regular expression
df['price'] = df['price'].str.replace(' USD|\\$', '', regex=True)

# Convert to numeric, raise error if unable to convert values
df['price'] = pd.to_numeric(df['price'], errors='raise')
print(f"{len(df['price'][df['price'] < p9999].dropna())}")
df = df[df['price'] < p9999].dropna(subset=['price'])

display(df[[ 'price']])

In [ ]:
def find_missing_rows(df1, df2):
    """
    Finds rows that exist in df1 but not in df2.
    This comparison is based on all columns.
    """
    # An outer merge aligns the two DataFrames. The indicator shows the source of each row.
    merged_df = df1.merge(df2, how='outer', indicator=True)
    
    # Filter for rows that are exclusive to the left DataFrame (df1).
    missing_rows_df = merged_df[merged_df['_merge'] == 'left_only'].drop(columns=['_merge'])
    print(f"Missing rows columns: {missing_rows_df.columns}, shape: {missing_rows_df.shape}")
    return missing_rows_df


In [ ]:
# Clean column names of any whitespace
df.columns = df.columns.str.strip()

# Remove columns, then rows that are not useful
# This will improve performance of operations on large datasets
df = df.drop(columns=['name', 'host_name', 'host_phone'])

df_before = df

# Remove latter occurrence of rows with duplicates in the 'id' column
df = df.drop_duplicates(subset=['id'])

display(find_missing_rows(df_before, df))


In [ ]:
# Convert to datetime and then to initial Unix timestamps (int64).
# Invalid dates become the integer -9223372037.
temp_series_int = pd.to_datetime(df['last_review'], errors='coerce').astype(np.int64) // 10**9

# To calculate the median correctly, make a float copy and replace the NaT integer with np.nan.
temp_series_for_median = temp_series_int.replace({-9223372037: np.nan})
# This calculates the median as a float.
median_timestamp = temp_series_for_median.median() 
# Go back to the original integer series and replace the NaT placeholder.
# Convert the calculated float median to an integer before replacing.
df['last_review'] = temp_series_int.replace({-9223372037: int(median_timestamp)})

display(df)

In [ ]:
# Clean 'room_type' column by removing leading/trailing whitespace and converting to lowercase
df['room_type'] = df['room_type'].str.strip().str.lower()
# Check the unique values in 'room_type' to ensure consistency
df['room_type'].value_counts()


In [ ]:


nomi = pgeocode.Nominatim('us')

def convert_zip_to_city(value, nomi):
    """
    Conditionally converts a zip code to a city name.
    - If the value is not a 5-digit number, it's returned as is.
    - If the value is a zip code, it's converted to a city name.
    """
    # Convert value to string to handle potential mixed types
    s_value = str(value).strip()

    # Check if the value is a 5-digit number (a likely zip code)
    if s_value.isdigit() and len(s_value) == 5:
        try:
            # Query for the location data of the zip code
            location_info = nomi.query_postal_code(s_value)
            
            # pgeocode returns NaN for invalid codes, so check for that
            if pd.isna(location_info.place_name):
                return value # Return original value if zip is not found

            city = location_info['place_name']
            zip_code = location_info['postal_code']

           
            # Conditional replacement for 'New York'
            if city == 'New York':
                # TODO: handle more cities with alternative names
                alternative_city_name = 'Manhattan'
                print(f"Converting zip code {zip_code} to city: {alternative_city_name}")
                return alternative_city_name
            elif city == 'Long Island City':
                alternative_city_name = 'Queens'
                print(f"Converting zip code {zip_code} to city: {alternative_city_name}")
                return alternative_city_name
            else:
                print(f"Converting zip code {zip_code} to city: {city}")
                return city
        except Exception:
            # If any error occurs during lookup, return the original value
            return value
    else:
        # If it's not a 5-digit number (e.g., "Queens", "Brooklyn"), return it unchanged
        return value

# Apply the updated function to the 'neighbourhood_group' column
df['neighbourhood_group'] = df['neighbourhood_group'].apply(lambda x: convert_zip_to_city(x, nomi))

df['neighbourhood_group'].value_counts()

## ⚙️ Part C: Feature Engineering

1. Create `log_price` as the log of the `price` column (use `np.log1p`)
2. One-hot encode `room_type` and `neighbourhood_group`
3. Create a new feature: `reviews_x_freq = number_of_reviews * reviews_per_month`
4. Optionally: Calculate days since `last_review`

In [ ]:
df['has_wifi'] = df['has_wifi'].astype(str).str.lower().isin(['true', 't', '1']).astype(int)
display(df[['id', 'has_wifi']])

In [ ]:
# Create a new column 'log_price' that contains the natural logarithm of 'price'
df['log_price'] = df['price'].apply(lambda x: np.log1p(x))
df = df.drop(['price'], axis=1)
display(df[['id', 'log_price']])

In [ ]:
# Convert reviews_per_month to numeric, coerce errors, then drop NaN values
df['reviews_per_month'] = pd.to_numeric(df['reviews_per_month'], errors='coerce').copy()
df = df.dropna(subset=['reviews_per_month'])
print(len(df['reviews_per_month']))

# create interaction feature. `reviews_x_freq`
df['reviews_x_freq'] = df['number_of_reviews'] * df['reviews_per_month']

display(df)

In [ ]:
df['has_wifi'] = df['has_wifi'].astype(str).str.lower().isin(['true', 't', '1']).astype(int)
display(df[['id', 'has_wifi']])

In [ ]:
	# Handle categorical features within the loop for each feature set
X = df.drop(columns=['id', 'log_price']).copy()
feature_set = X.columns.tolist()

# Identify categorical and numerical features for the current set
numerical_features = [column for column in X.columns if np.issubdtype(X[column].dtype, np.number)]
# Identify categorical features.
categorical_features = [column for column in X.columns if X[column].dtype in ['object', 'category']]
display(f"Numerical features: {numerical_features}")
display(f"Categorical features: {categorical_features}")



# Create the column transformer for one-hot encoding
preprocessor = ColumnTransformer(
		transformers=[
				('num', 'passthrough', numerical_features),
				('cat', OneHotEncoder(handle_unknown='error'), categorical_features)])

## ✂️ Part D: Train/Test Split

Use `train_test_split` to divide your data:
- 80% training
- 20% testing

Select relevant features for modeling (no ID, name, or original price).

In [ ]:
display(X)

## 📈 Part E: Linear Regression Modeling

1. Train a `LinearRegression` model to predict `log_price`
2. Evaluate it on both train and test sets using:
   - RMSE (Root Mean Squared Error)
   - R² (coefficient of determination)
3. Print model coefficients
4. Interpret: Which features have the strongest impact?

In [ ]:
# Create the model pipeline
# Add a normalizer

model = Pipeline(steps=[('preprocessor', preprocessor),
												# ('scaler', MinMaxScaler()),
												('regressor', LinearRegression())])
y = df['log_price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train the model using the training data
model.fit(X_train, y_train)
# Make predictions on the test data
y_pred = model.predict(X_test)
# Calculate R2 score
r2 = r2_score(y_test, y_pred)
display(f"R2 score: {r2:.4f}")
display(f"RMSE: {root_mean_squared_error(y_test, y_pred):.4f}")


In [ ]:

print(f"Fixed features shape: {X.shape}")
print(f"Remaining features: {X.columns.tolist()}")

# Add days_since_last_review feature
current_timestamp = int(time.time())
df['days_since_last_review'] = (current_timestamp - df['last_review']) / 86400  # Convert seconds to days

print(f"Days since last review - Min: {df['days_since_last_review'].min():.1f}, Max: {df['days_since_last_review'].max():.1f}")

# Update feature sets after adding new feature
X = df.drop(columns=['log_price', 'id']).copy()
numerical_features = [column for column in X.columns if np.issubdtype(X[column].dtype, np.number)]
categorical_features = [column for column in X.columns if X[column].dtype in ['object', 'category']]

print(f"Updated numerical features: {numerical_features}")
print(f"Updated categorical features: {categorical_features}")

In [ ]:
# FIXED PIPELINE: Remove MinMaxScaler and use corrected features
from sklearn.model_selection import cross_validate

# Create the corrected preprocessor with updated feature lists
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_features),
        ('cat', OneHotEncoder(handle_unknown='error'), categorical_features)
    ]
)

# Create the corrected pipeline without MinMaxScaler
pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                           ('regressor', LinearRegression())])

y = df['log_price']

# Split the data into training and testing sets with corrected X
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Cross-validation for robust evaluation
scoring = {
    'RMSE': 'neg_root_mean_squared_error',
    'R2': 'r2'
}

cv_results = cross_validate(pipeline, X_train, y_train, cv=5, scoring=scoring)

# The RMSE scores are negative, so we multiply by -1 to make them positive
mean_cv_rmse = -np.mean(cv_results['test_RMSE'])
mean_cv_r2 = np.mean(cv_results['test_R2'])

print(f"Mean CV RMSE: {mean_cv_rmse:.4f}")
print(f"Mean CV R-squared: {mean_cv_r2:.4f}")

# Fit the pipeline on the entire training set
pipeline.fit(X_train, y_train)

# Make predictions on the test set
y_pred = pipeline.predict(X_test)

# Calculate and print the performance metrics for the test set
test_rmse = root_mean_squared_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)

print("--- Test Set Performance ---")
print(f"Test RMSE: {test_rmse:.4f}")
print(f"Test R-squared: {test_r2:.4f}")

In [ ]:
# Feature Coefficient Analysis
# Get the one-hot encoder from the pipeline's preprocessor step
one_hot_encoder = pipeline.named_steps['preprocessor'].named_transformers_['cat']

# Get the feature names after one-hot encoding
encoded_feature_names = one_hot_encoder.get_feature_names_out(categorical_features)

# Combine the original numerical feature names with the new encoded categorical names
all_feature_names = numerical_features + list(encoded_feature_names)

# Get the coefficients from the trained linear regression model
coefficients = pipeline.named_steps['regressor'].coef_

# Create a DataFrame to view the coefficients alongside their feature names
coef_df = pd.DataFrame({'Feature': all_feature_names, 'Coefficient': coefficients})

# Sort by the absolute value of coefficients to see which features have the most impact
coef_df['Abs_Coefficient'] = coef_df['Coefficient'].abs()
coef_df = coef_df.sort_values(by='Abs_Coefficient', ascending=False).drop(columns=['Abs_Coefficient'])

print("Feature Coefficients:")
display(coef_df)

## 🧠 Bonus (Optional)

Try using `PolynomialFeatures` to add interaction terms between some variables.

Does the model improve? Or overfit?


In [ ]:
# PolynomialFeatures Implementation
from sklearn.preprocessing import PolynomialFeatures

print("=== BASELINE MODEL PERFORMANCE ===")
print(f"Baseline R²: {test_r2:.4f}")
print(f"Baseline RMSE: {test_rmse:.4f}")

# Create polynomial features using only numerical features (avoiding categorical explosion)
numerical_only = ['minimum_nights', 'number_of_reviews', 'reviews_per_month', 'has_wifi', 'reviews_x_freq', 'days_since_last_review']

print(f"\nNumerical features for polynomial: {numerical_only}")
print(f"Shape of numerical features: {X[numerical_only].shape}")

# Create polynomial pipeline with interaction terms only
poly_preprocessor = ColumnTransformer(
    transformers=[
        ('poly', PolynomialFeatures(degree=2, interaction_only=True, include_bias=False), numerical_only),
        ('cat', OneHotEncoder(handle_unknown='error'), categorical_features)
    ]
)

polynomial_pipeline = Pipeline(steps=[
    ('preprocessor', poly_preprocessor),
    ('regressor', LinearRegression())
])

print("\n=== TRAINING POLYNOMIAL MODEL ===")
polynomial_pipeline.fit(X_train, y_train)
y_pred_poly = polynomial_pipeline.predict(X_test)

# Calculate performance metrics
poly_r2 = r2_score(y_test, y_pred_poly)
poly_rmse = root_mean_squared_error(y_test, y_pred_poly)

print(f"Polynomial R²: {poly_r2:.4f}")
print(f"Polynomial RMSE: {poly_rmse:.4f}")

# Cross-validation comparison
scoring = {'RMSE': 'neg_root_mean_squared_error', 'R2': 'r2'}

baseline_cv = cross_validate(pipeline, X_train, y_train, cv=5, scoring=scoring)
poly_cv = cross_validate(polynomial_pipeline, X_train, y_train, cv=5, scoring=scoring)

baseline_cv_r2 = np.mean(baseline_cv['test_R2'])
poly_cv_r2 = np.mean(poly_cv['test_R2'])

print("\n=== CROSS-VALIDATION COMPARISON ===")
print(f"Baseline CV R²: {baseline_cv_r2:.4f}")
print(f"Polynomial CV R²: {poly_cv_r2:.4f}")
print(f"Improvement: {poly_cv_r2 - baseline_cv_r2:.4f}")

if poly_cv_r2 - baseline_cv_r2 > 0.05:
    print("✅ SIGNIFICANT IMPROVEMENT: Polynomial features help!")
elif poly_cv_r2 - baseline_cv_r2 > 0:
    print("⚠️  MARGINAL IMPROVEMENT")  
else:
    print("❌ NO IMPROVEMENT: Possible overfitting")

## ✅ Submission Checklist
- [ ] Notebook is clean and readable
- [ ] You handled weird or dirty data
- [ ] You performed EDA
- [ ] You trained and evaluated a regression model
- [ ] You wrote clear comments or markdown explanations